In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, ttest_ind, pearsonr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import env
import acquire
import prep
import split_scale
import model

Data Dictionary located in README.md 

# Plan
- As a junior data scientist, an email requesting the following is recieved?:
    1. Predict the values of single unit properties that a tax district assesses using property data from whos last transaction was during the "hot months" (in terms of real estate demand) of May and June 2017.
    
    2. A few problems exist:    
    __a:__ the current data which gave the location of each property has been lost. We need to know what county each property is located in. So we will have to find a way to acquire this data.
    
    3. The Zillow Data Science team would also like to know the distribution of tax rates for each county but specified this is not part of the __MVP__
        - since the data already has the tax amounts and tax value of the home it should be easy to create a new notebook with this information. 
        __We cannot use this information in our model__

# Deliverables
1. Create a report in the form of slides and present it verbally.
2. Create a github repository containing all my work, which should consist of the following:
    - at least 1 jupyter notebook that walks throuh the pipeline
    - ensure all questions are being answered 
    - add all the `.py` files so our work can be reproduced, knowing that for this to be reporduced by someone else they would have to have their own `env.py` so they can access the SQL  Zillow database

# Aquire  

> Our first iteration will be to use a SQL query to pull data into a jupyter notebook and create a dataframe. Using the following features: square feet of the home, number of bedrooms, number of bathrooms to estimate value of __taxvaslueddollarcnt__. This will be our __MVP(minimally viable product)__

## An MVP is information that is collected and validated with the least effort. The primary benefit of an MVP is to satisfy the request of the Zillow team with just the features they have requested. 

In [2]:
# Here is the SQL query used
query = """ SELECT
    prop.parcelid,
    bathroomcnt AS bathrooms,
    bedroomcnt AS bedrooms,
    calculatedfinishedsquarefeet AS square_feet,
    fips AS fips_number,
    ptype.propertylandusetypeid,
    ptype.propertylandusedesc,
    taxvaluedollarcnt AS home_value,
    taxamount AS tax_amount
    FROM properties_2017 AS prop
        JOIN
        predictions_2017 AS pred
        ON prop.parcelid = pred.parcelid
        JOIN
        propertylandusetype AS ptype
        ON prop.propertylandusetypeid = ptype.propertylandusetypeid

    WHERE transactiondate
    BETWEEN '2017-05-01' AND '2017-06-30' AND propertylandusedesc = ('Single Family Residential')"""

# pull clean data from previous notebook

df = acquire.get_zillow_data_from_sql()

## Pull data into jupyter notebook and create a Pandas DataFrame
  

In [3]:
df.head()

,parcelid,bathrooms,bedrooms,square_feet,fips_number,propertylandusetypeid,propertylandusedesc,home_value,tax_amount
0,11289917,2.0,3.0,1458.0,6037.0,261,Single Family Residential,136104.0,2319.90
1,11705026,1.0,2.0,1421.0,6037.0,261,Single Family Residential,35606.0,543.69
2,14269464,3.0,4.0,2541.0,6059.0,261,Single Family Residential,880456.0,9819.72
3,11389003,2.0,3.0,1650.0,6037.0,261,Single Family Residential,614000.0,7673.19
4,11967869,1.0,2.0,693.0,6037.0,261,Single Family Residential,274237.0,3267.47


# Prepare Date
    

## Dive inte the data

In [4]:
df.head()

,parcelid,bathrooms,bedrooms,square_feet,fips_number,propertylandusetypeid,propertylandusedesc,home_value,tax_amount
0,11289917,2.0,3.0,1458.0,6037.0,261,Single Family Residential,136104.0,2319.90
1,11705026,1.0,2.0,1421.0,6037.0,261,Single Family Residential,35606.0,543.69
2,14269464,3.0,4.0,2541.0,6059.0,261,Single Family Residential,880456.0,9819.72
3,11389003,2.0,3.0,1650.0,6037.0,261,Single Family Residential,614000.0,7673.19
4,11967869,1.0,2.0,693.0,6037.0,261,Single Family Residential,274237.0,3267.47


In [5]:
df.dtypes

parcelid                   int64
bathrooms                float64
bedrooms                 float64
square_feet              float64
fips_number              float64
propertylandusetypeid      int64
propertylandusedesc       object
home_value               float64
tax_amount               float64
dtype: object

In [13]:
df['bathrooms'].value_counts().sum()

15036

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
print(f'My df has {df.shape[0]} rows and {df.shape[1]} columns.')

## Prep the data
  - df.isnull().sum()
  - create a prep.py file to use in cleaning the data
  - use function `clean_data` from `prep.py` to drop nulls
  - I chose to set the index to parcel id because each property has a unqiue number
  - plot the distributions of the independent variables
      - Independent variables are controlled inputs and we will use these types of variables to study the effect they have on the dependent variable

In [ ]:
df.isnull().sum()

__It is important to use different variables when changing the dataframe to ensure you are using the correct data__

In [ ]:
df_clean = prep.clean_data(df)
## verify nulls are gone
df_clean.isnull().sum()

In [ ]:
df_clean.info()

In [ ]:
# we need to change some data types and we also need to set an index id sp for the
# writing I am changing df_clean back to the variable name df
df = df_clean .set_index(['parcelid'])

# changing data types

df = df.astype({'propertylandusedesc': 'category',
               'fips_number': 'int'})

df.info()

In [ ]:
# add the county for each property based off the fips_number which came from o
# utside the dataframe. This tells what county each property is located in
df['county_name'] = df.fips_number.map({6037: 'Los Angeles', 
                                        6059: 'Orange',
                                        6111: 'Ventura'
                                       })

In [ ]:
df.head()

In [ ]:
# need to drop some columns 
df = df.drop(columns=['fips_number', 'propertylandusetypeid', 'propertylandusedesc',
                 'tax_amount', 'county_name'])

# Data Explore & Preprocess

1. Visualize Attributes and interactions
2. Anlyze: using statsmodels, numpy, scipy, scikit-learn
3. __Possible Deliverable Product__
    - report of analysis
    - presentation slide
4. Feature engineering
    - Zillow has chosen the features for us:
        - square feet of home
        - number of bedrooms
        - number of bathrooms
5. Summarize our takaways and conclusions

In [ ]:
sns.pairplot(df, corner=True)

In [ ]:
sns.pairplot(df, corner=True)

In [ ]:
df.hist()

In [ ]:
df.corr()

In [ ]:
# plot out the correlations w/ a heatmap to get visual insight
plt.title("Heatmap of columns")
sns.heatmap(df.corr(), cmap='Blues', annot=True)
plt.show()

## What can we see from this heatmap

- our dependent variable is home_value and there is a correlation between `bathrooms`, `bedrooms`, and `square_feet`
- the feature with the highest coorelation is `square_feet`

In [ ]:
import sklearn.model_selection

# split into train and test
train, test = sklearn.model_selection.train_test_split(df, train_size=.8, random_state=123)

In [ ]:
predictions = pd.DataFrame({'actual' : train.home_value })

In [ ]:
predictions.head()

In [ ]:
# Model 1
X = train[['bathrooms']]
y = train.home_value

lm_1 = sklearn.linear_model.LinearRegression()
lm_1.fit(X, y)
predictions['bathrooms'] = lm_1.predict(X)

In [ ]:
lm_1.coef_, lm_1.intercept_

In [ ]:
predictions.head()

In [ ]:
X = train[['bedrooms']]
y = train.home_value

lm_2 = sklearn.linear_model.LinearRegression()
lm_2.fit(X, y)
predictions['bedrooms'] = lm_2.predict(X)

In [ ]:
lm_2.coef_, lm_2.intercept_

In [ ]:
predictions.head()

In [ ]:
X = train[['square_feet']]
y = train.home_value

lm_3 = sklearn.linear_model.LinearRegression()
lm_3.fit(X, y)
predictions['square_feeet'] = lm_3.predict(X)

In [ ]:
lm_3.coef_, lm_3.intercept_

In [ ]:
predictions.head()

In [ ]:
sns.pairplot(predictions, kind = 'reg')

In [ ]:
plt.title("Heatmap of columns")
sns.heatmap(predictions.corr(), cmap='Blues', annot=True)
plt.show()

In [ ]:
# Define X and y variables
X = df[['bathrooms', 'bedrooms', 'square_feet']]
y = df[['home_value']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75, random_state=123)

In [ ]:
X_train.head()

In [ ]:
results = smf.ols('home_value ~ bedrooms + bathrooms + square_feet', data=df).fit()

In [ ]:
print(results.summary())

In [ ]:
df.head()

In [ ]:
# Create Model
ols_model = ols(formula='home_value ~ bedrooms + bathrooms + square_feet', data=df).fit()

# Predict Model
ols_yhat = ols_model.predict(X_train)

In [ ]:
# Create a DataFrame for evaluating my model(s) and Baseline Value
ols_eval = y_train.copy()
ols_eval.rename(columns={'home_value': 'actual'}, inplace=True)

In [ ]:
# Add Baseline Column
ols_eval.rename(columns={'home_value': 'actual'}, inplace=True)

In [ ]:
# Add Baseline
ols_eval['baseline_yhat'] = ols_eval['actual'].mean()

In [ ]:
# Add OLS predictions columns
ols_eval['ols_yhat'] = ols_model.predict(X_train)

In [ ]:
# Calculate and Add residuals colomns for plotting
ols_eval['residuals'] = ols_eval.ols_yhat - ols_eval.actual

In [ ]:
# Compute the RMSE for our ols Model and Baseline using our created dataframe
baseline_RMSE = sqrt(mean_squared_error(ols_eval.actual, ols_eval.baseline_yhat))

ols_RMSE = sqrt(mean_squared_error(ols_eval.actual, ols_eval.ols_yhat))
print(baseline_RMSE)
print(ols_RMSE)
print(f'My model has value: {ols_RMSE < baseline_RMSE}')

In [ ]:
# Compute the RMSE for the model we created
ols_r2 = round(ols_model.rsquared,3)

ols_p_value = ols_model.f_pvalue

print(f'My R-squared score is significant: {ols_p_value < .05}')

In [ ]:
ols_eval.head()

In [ ]:
print(baseline_RMSE), print(ols_RMSE)

In [ ]:
model.plot_residuals(ols_eval.actual, ols_eval.ols_yhat)

In [ ]:
# create a histogram 
plt.hist(np.log(ols_eval.residuals))

In [ ]:
# create a scatter plot of the residuals and look for patters
plt.scatter(ols_eval.actual, ols_eval.residuals)

In [ ]:
# look at predictions vs residuals
plt.scatter(ols_eval.ols_yhat, ols_eval.residuals)

In [ ]:
df.head()

In [ ]:
# Define X and y variables
X = df[['bathrooms', 'bedrooms', 'square_feet']]
y = df[['home_value']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75, random_state=123)

In [ ]:
X_train.head()

In [ ]:
# Initialize the Linear Regression Object 
lm = LinearRegression()

rfe = RFE(lm, 2)

# Transforming data using RFE
X_rfe = rfe.fit_transform(X_train,y_train)

In [ ]:
# boolean mask for each variable of whether it was selected or not. 
mask = rfe.support_

# select the column names of the features that were selected and convert them to a list for future use. 
rfe_features = X_train.columns[mask]

# print them out here for our reference
print(f'selected {len(rfe_features)} features:', ', '.join(rfe_features))

In [ ]:
lm = LinearRegression()
lm

In [ ]:
# Fitting the data to model
lm.fit(X_rfe, y_train)

In [ ]:
print("Linear Model:", lm)

print("intercept: ", lm.intercept_)

print("features: ", rfe_features)
print("coefficients: ", lm.coef_)

In [ ]:
y_train['yhat_lm'] = lm.predict(X_rfe)

y_train.head()

In [ ]:
# Evaluate RMSE
RMSE_lm = np.sqrt(mean_squared_error(y_train.home_value, y_train.yhat_lm))
RMSE_lm

print("linear model\n  Root mean squared error: {:.3}".format(RMSE_lm))

In [ ]:
r2_lm = lm.score(X_rfe, y_train.home_value)

print(f"{r2_lm:.2%} of the variance in the home's value can be explained by the difference in bedrooms and bathrooms.")

print("This means almost 73% of the variance in the home's value is due to other factors, such as square feet.")
print("Previous models have shown that square footage has the highest correlation to home value.")
      

In [ ]:
output = "{} = {:.4} + {:.2} * {} + {:.3} * {}".format(
    y_train.columns[0],
    lm.intercept_[0],
    lm.coef_[0][0],
    rfe_features[0],
    lm.coef_[0][1],
    rfe_features[1],
)
print(output)

# Poly Nomial Regression Model

In [ ]:
# Poly Nomial Regression model

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_rfe)

poly.get_feature_names()

In [ ]:
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_train.home_value)
y_train['yhat_poly'] = lm_poly.predict(X_poly)

In [ ]:
RMSE_poly = np.sqrt(mean_squared_error(y_train.home_value, y_train.yhat_poly))
RMSE_poly

print("polynomial model\n  Root mean squared error: {:.3}".format(RMSE_poly))

In [ ]:
y_train.head()

In [ ]:
# set predictions to be the mean of all final grades
y_train['yhat_baseline'] = df['home_value'].mean()

# compute the RMSE
RMSE_bl = np.sqrt(mean_squared_error(y_train.home_value, y_train.yhat_baseline))
print("Baseline (ŷ = ȳ)\n  Root mean squared error: {:.3}".format(RMSE_bl)) 

# no need to compute R-2 because it will be a 0! But we will demonstrate here:
evs = explained_variance_score(y_train.home_value, y_train.yhat_baseline)
print('  {:.2%} of the variance.'.format(evs))

In [ ]:
y_train.head()

In [ ]:
plt.figure(figsize=(9, 9))

plt.scatter(y_train.home_value, y_train.yhat_lm, label='OLS (home_value ~ bedrooms + bathrooms + square_feet)', marker='o')
plt.scatter(y_train.home_value, y_train.yhat_poly, label='Model with polynomial features', marker='o')
plt.scatter(y_train.home_value, y_train.yhat_baseline, label=r'Baseline ($\hat{y} = \bar{y}$)', marker='o')
plt.plot([60, 100], [60, 100], label='Perfect predictions', ls=':', c='grey')

plt.legend(title='Model')
plt.ylabel('Predicted Home Value')
plt.xlabel('Actual Home Value')
plt.title('Predicted vs Actual Home Value')